In [1]:
import tensorflow as tf
import os
import net
from tensorflow.examples.tutorials.mnist import input_data

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVIE_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "."
MODEL_NAME = "model.ckpt"

In [3]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, net.INPUT_NODE], name = "x-input")
    y_ = tf.placeholder(tf.float32, [None, net.OUTPUT_NODE], name = "y-input")
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = net.net(x, regularizer)
    
    global_step = tf.Variable(0,trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVIE_AVERAGE_DECAY, global_step)
    
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y , labels=tf.argmax(y_, 1))
    
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, 
                                               global_step, 
                                               mnist.train.num_examples/ BATCH_SIZE,
                                               LEARNING_RATE_DECAY
                                               )
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')  
        #確保train_step, variable_averages_op有執行,tf.no_op此步驟沒做事
        
#     correct_prediction = tf.equal(tf.argmax(y, 1) , tf.argmax(y_, 1))
    
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAINING_STEPS):
            xs , ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict = {x : xs , y_ : ys})
            
            if i%1000 == 0:
                print("After %d training step , loss on training batch is %g."% (step, loss_value))
                saver.save(sess , os.path.join(MODEL_SAVE_PATH,MODEL_NAME), global_step = global_step)

In [4]:
def main(argv=None):
    mnist = input_data.read_data_sets("./test_data" , one_hot=True)
    train(mnist)
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./test_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./test_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./test_data/t10k-images-idx3-ubyte.gz
Extracting ./test_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 1 training step , loss on training batch is 2.93463.
After 1001 training step , loss on training batch is 0.182076.
After 2001 training step , loss on training batch is 0.159506.
After 3001 training step , loss on training batch is 0.168977.
After 4001 training step , loss on training batch

SystemExit: 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
